# Dataset Preprocessing

The following steps were taken during the dataset preprocessing:
- Columns "name" were renamed into "author_name" and "recipe_name" to eliminate any confusion. 
- During manual inspection, it was found that "author_note" column had hashtags and html tags in it. The html tags were removed using BeautifulSoup across all columns, along with the hashtags.
- "Steps" column contained unstructured text that included clearly separated steps. During the manual inspection, it was found that the steps are separated either with ",." or new lines. New lines in the dataset were respresented with:
    - *\n* or *\r\n*
    - *\n\n* or *\r\n\r\n*  
    
    Based on compilation environment. Newlines were replaced with *\n* to standardize the dataset.

    While single new lines appear to function as "sub steps" separators in some cases, it is still better to divide on them, to make steps more granular. Then, we evaluate whether each row of the dataset contains either of separators, to ensure that any other separators are not present. The output is were as follows: 
    ```
    Number of rows in column: 100
    Number of rows with '.,' separator: 100
    Number of rows with '\n\n' separator: 54
    Number of rows with '\n' separator: 59
    Number of rows with either ',.' or newlines: 100
    ```
    Concluding that other separators were not missed during manual inspection, the strings in the "steps" column were separated into lists. 

# Data Indexing and Vector Store Creation
- Embedding model was chosen by filtering out open-source models that support multilinguality in [MTEB Leaderboard](https://huggingface.co/spaces/mteb/leaderboard). 
- The dataset from the csv file was imported into a list of documents using LangChain's `CSVLoader`, constructing one document per recipe/row. 
- To determine whether documents should be further splitted into text chunks, analysis of their lengths was conducted: 
    ```
    Maximum document length: 1344 characters
    Found in document index: 55
    =====================
    recipe_name: Kuře z jednoho hrnce
    author_name: Roman Vaněk
    ingredients: Česnek, Chilli koření mleté, Cuketa, Hřebíček mletý, Koriandr mletý, Krájená cibule, Kuřecí vykostěná stehna, Kuřecí vývar, Kurkuma mletá, Med, Mrkev, Olivový olej, Pepř černý , Plocholistá petržel, Pomerančová šťáva, Římský kmín mletý, Rýže, Skořice mletá, Sladká paprika, Sůl, Těstoviny, Víno vhodné k drůbežímu masu, Zázvor
    steps: ['Do kastrolu přidáme cuketu a mrkev, promícháme a pečeme 2 minuty. Občas zamícháme. Přidáme cibuli, promícháme a pečeme 1 minutu.', 'Kastrol vložíme do rozehřáté trouby a pečeme 40 minut. Před podáváním přimícháme nakrájenou petržel.', 'Ke koření přilijeme pomerančovou šťávu a vývar, přidáme med a pomerančovou kůru a promícháme. Přivedeme k varu a nalijeme do kastrolu ke kuřeti.', 'Maso vydatně osolíme a opepříme z obou stran.', 'Mrkev nakrájíme na kolečka silná 0,5 cm. Cibuli nakrájíme najemno. Česnek nakrájíme na plátky. Cuketu rozkrojíme podélně na čtvrtiny a ty pak nakrájíme na plátky silné 0,5 cm. Zázvor nastrouháme a šťávu z něj vymačkáme na mrkev.', 'Přidáme česnek a zázvor, promícháme a za občasného míchání pečeme 2 minuty. Kastrol stáhneme z ohně a vyskládáme do něj opečené maso kůží nahoru.', 'Troubu rozehřejeme na 180 °C.', 'V kastrolu rozpálíme na vyšším ohni olej a maso na něm opečeme 4 minuty dozlatova na s.']
    ```

    1344 characters is far from any embedding model's context limit, so no further splitting was performed.
